### Backtest

In [ ]:
import os
import vectorbt as vbt
import pandas as pd
import numpy as np
from pandas_ta import ema

Pay attention to the following block of code so the strategy settings will be inputed correctly:

In [ ]:
# MANUAL INPUTS:
inp_start='2023-01-01 00:00:00.000' # start backetest time
inp_end='2023-12-31 23:59:59.999' # end backtest time

inp_tf = "5min" # timeframe for ema

deposit = 300 # in USDT
comission = 0.2 # already in %
leverage = 1

MaintenanceMarginRate = 0.5 # already in %: https://www.binance.com/ru/futures/trading-rules/perpetual/leverage-margin
MaintenanceAmountLong = 50
MaintenanceAmountShort = 50

show_real_drawdown = False # True or False. If True,calcualtion will be much slower
plot_results = True # True or False to show plots
print_data = True # True or False to show data

inp_ema_len = 2 # length of ema

order_1_long = 2.17 # entry from total deposit in %
order_2_long = 4.35  #entry from total deposit in %
order_3_long = 8.7 # entry from total deposit in %
order_4_long = 18.12 # entry from total deposit in %
order_5_short = 66.67 # entry from total deposit in %

long_1_dca_per = 0.5 # DCA down from 1st entry, already in %
long_2_dca_per = 1.0 # DCA down from 1st entry, already in %
long_3_dca_per = 1.5 # DCA down from 1st entry, already in %
long_4_dca_per = 2.0 # DCA down from 1st entry, already in %
short_5_dca_per = 2.5 # DCA down from 1st entry, already in %

tp_long = 0.6 # Take Profit Long,already in %
tp_short = 1.6 # Take Profit Short, already in %

inp_ticker = 'CRVUSDT' # asset name
inp_round_price = 3 # every asset has different number of decimals in price, check asset at Binance for details
inp_round_qty = 1 # every asset has different number of decimals in quantity, check asset at Binance for details

inp_strategy_name = 'Hedge Scalp' # name of strategy

In [ ]:
# Set up the Vectorbt settings
vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1300
# vbt.settings['plotting']['layout']['height'] = 800

# Set up pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Get the path of the directory to cleaned data:
dir = os.getcwd()
data = dir+f'/cleaned_data/{inp_ticker}.feather'

In [ ]:
# Read tick data from cleaned_data:"""
tick = pd.read_feather(data)

tick=tick[(tick['time'] >= inp_start) & (tick['time'] < inp_end)]
tick = tick.loc[:,['time','price']]
tick = tick.rename(columns={'time':'Time','price':'close_tick',})
tick['Time'] = pd.to_datetime(tick['Time'],unit='ms')#.dt.floor('s')
tick = tick.set_index(tick['Time'])

if print_data:
    print('\n'+('-'*5)+'Tick Data (head)'+('-'*5))
    print(tick.head(10))
    print(tick.shape)

In [ ]:

# Resample tick data using predefined timeframe:
tick_resampled_close = tick['close_tick'].resample(inp_tf).last()
tick_resampled_close, _ = tick_resampled_close.align(tick['close_tick'], join="left")
tick_resampled_close = tick_resampled_close.rename('close_resampled')

tick_resampled_open = tick['close_tick'].resample(inp_tf).first()
tick_resampled_open, _ = tick_resampled_open.align(tick['close_tick'], join="left")
tick_resampled_open = tick_resampled_open.rename('open_resampled')

tick_resampled_high = tick['close_tick'].resample(inp_tf).max()
tick_resampled_high, _ = tick_resampled_high.align(tick['close_tick'], join="left")
tick_resampled_high = tick_resampled_high.rename('high_resampled')

tick_resampled_low = tick['close_tick'].resample(inp_tf).min()
tick_resampled_low, _ = tick_resampled_low.align(tick['close_tick'], join="left")
tick_resampled_low = tick_resampled_low.rename('low_resampled')

tick_resampled = pd.concat([tick_resampled_close, tick_resampled_open, tick_resampled_high, tick_resampled_low], axis="columns")
tick_resampled[['close_resampled','open_resampled','high_resampled','low_resampled']]
tick_resampled = tick_resampled.reset_index()
del tick_resampled_close, tick_resampled_open, tick_resampled_high, tick_resampled_low

tick_resampled['ema'] = round((ema(close=tick_resampled['close_resampled'],length=inp_ema_len)),inp_round_price).shift(1) #type:ignore
tick_resampled['grid_1_long'] = round((tick_resampled['ema']-(tick_resampled['ema'] * long_1_dca_per/100)),inp_round_price)
tick_resampled['grid_2_long'] = round((tick_resampled['ema']-(tick_resampled['ema'] * long_2_dca_per/100)),inp_round_price)
tick_resampled['grid_3_long'] = round((tick_resampled['ema']-(tick_resampled['ema'] * long_3_dca_per/100)),inp_round_price)
tick_resampled['grid_4_long'] = round((tick_resampled['ema']-(tick_resampled['ema'] * long_4_dca_per/100)),inp_round_price)
tick_resampled['grid_5_short'] = round((tick_resampled['ema']-(tick_resampled['ema'] * short_5_dca_per/100)),inp_round_price)

tick_resampled = tick_resampled[['Time','close_resampled','open_resampled','high_resampled','low_resampled','ema','grid_1_long','grid_2_long','grid_3_long','grid_4_long','grid_5_short']] #selecting colums I need

if print_data:
    print('\n'+('-'*5)+inp_tf+' Data (tail)'+('-'*5))
    print(tick_resampled.tail(10))
    print(tick_resampled.shape)

In [ ]:
# Concat original tick data and resampled tick data, creating new data frame:
tick = tick.reset_index(drop=True)
tick_resampled = tick_resampled.reset_index(drop=True)

df = pd.concat([tick, tick_resampled],sort=False)
df = df.sort_values(by=['Time'], ascending=True)

df = df.set_index(df['Time'])
df = df[['Time','close_tick','close_resampled','open_resampled','high_resampled','low_resampled','ema','grid_1_long','grid_2_long','grid_3_long','grid_4_long','grid_5_short']]
df = df.ffill() # to fill NaN by last available value

df['close_tick_shifted'] = df['close_tick'].shift(2)
df['grid_1_long_shifted'] = df['grid_1_long'].shift(1)
df['signal_long'] = np.where(((df['close_tick']<df['grid_1_long']) & (df['close_tick_shifted']>df['grid_1_long_shifted'])),1,0)

if print_data:
    print('\n'+('-'*5)+'Final Data (head)'+('-'*5))
    print(df.head(10))
    print('\n'+('-'*5)+'Final Data (tail)'+('-'*5))
    print(df.tail(10))

In [ ]:

# Strategy:
trades = pd.DataFrame(columns=['time','close_tick','pos','side','position_qty_long','tp_target_long','tp_quantity_long','position_qty_short','tp_target_short','tp_quantity_short','cumulative_profit_long','cumulative_profit_short','cumulative_profit','avg_price_long','avg_price_short'])

long_1_taken = False
long_2_taken = False
long_3_taken = False
long_4_taken = False
tp_long_taken = False

short_5_taken = False
tp_short_taken = False

entry_price_1_long = 0
entry_price_2_long = 0
entry_price_3_long = 0
entry_price_4_long = 0
tp_price_long = 0

dca_grid_2_long = 0
dca_grid_3_long = 0
dca_grid_4_long = 0
dca_grid_5_short = 0

entry_price_5_short = 0
tp_price_short = 0

position_qty_long_1 = 0
position_qty_long_2 = 0
position_qty_long_3 = 0
position_qty_long_4 = 0

position_qty_short_5 = 0

close_resampled = 0

avg_price_long = 0
avg_price_short = 0

tp_target_long = 0
tp_target_short = 0

cumulative_profit_long = 0
cumulative_profit_short = 0
cumulative_profit = 0

tp_quantity_long = 0
tp_quantity_short = 0

pnl_sum = 0
pnl_long = 0
pnl_short = 0

for i in df.itertuples():

    if long_1_taken == False: # Long 1
        if i.signal_long==1:
            entry_price_1_long = round(i.close_tick, inp_round_price)
            dca_grid_2_long = i.grid_2_long
            dca_grid_3_long = i.grid_3_long
            dca_grid_4_long = i.grid_4_long
            dca_grid_5_short = i.grid_5_short

            position_qty_long_1 = round((deposit*order_1_long/100*leverage)/(entry_price_1_long), inp_round_qty)

            tp_quantity_long = position_qty_long_1
            tp_target_long = round((entry_price_1_long*(1+(tp_long/100))), inp_round_price)

            avg_price_long = entry_price_1_long

            new_row = [i.Time, entry_price_1_long,1,'Long 1',position_qty_long_1, tp_target_long, tp_quantity_long,0,0,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
            # logger_api.info(str(new_row))
            trades.loc[len(trades)] = new_row # type: ignore
            long_1_taken = True

    if long_1_taken == True: # Long 2
        if long_2_taken == False:
            if i.close_tick<dca_grid_2_long:
                entry_price_2_long = round(i.close_tick, inp_round_price)

                position_qty_long_2 = round((deposit*order_2_long/100*leverage)/(entry_price_2_long), inp_round_qty)

                tp_quantity_long = round(position_qty_long_1+position_qty_long_2, inp_round_qty)
                tp_target_long = round((((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2))/tp_quantity_long)*(1+tp_long/100), inp_round_price)

                avg_price_long = ((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2))/tp_quantity_long
                pnl_long = ((i.close_tick - avg_price_long)*tp_quantity_long)*(1-comission/100)

                new_row = [i.Time, entry_price_2_long,1,'Long 2', position_qty_long_2, tp_target_long, tp_quantity_long,0,0,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
                trades.loc[len(trades)] = new_row # type: ignore
                long_2_taken = True

    if long_2_taken == True: # Long 3
        if long_3_taken == False:
            if i.close_tick<dca_grid_3_long:
                entry_price_3_long = round(i.close_tick, inp_round_price)

                position_qty_long_3 = round((deposit*order_3_long/100*leverage)/(entry_price_3_long), inp_round_qty)

                tp_quantity_long = round(position_qty_long_1+position_qty_long_2+position_qty_long_3, inp_round_qty)
                tp_target_long = round((((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3))/tp_quantity_long)*(1+tp_long/100), inp_round_price)

                avg_price_long = ((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3))/tp_quantity_long
                pnl_long = ((i.close_tick - avg_price_long)*tp_quantity_long)*(1-comission/100)

                new_row = [i.Time, entry_price_3_long,1,'Long 3', position_qty_long_3, tp_target_long, tp_quantity_long,0,0,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
                trades.loc[len(trades)] = new_row # type: ignore
                long_3_taken = True

    if long_3_taken == True: # Long 4
        if long_4_taken == False:
            if i.close_tick<dca_grid_4_long:
                entry_price_4_long = round(i.close_tick, inp_round_price)

                position_qty_long_4 = round((deposit*order_4_long/100*leverage)/(entry_price_4_long), inp_round_qty)

                tp_quantity_long = round(position_qty_long_1+position_qty_long_2+position_qty_long_3+position_qty_long_4, inp_round_qty)
                tp_target_long = round((((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long)*(1+tp_long/100), inp_round_price)

                avg_price_long = ((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long
                pnl_long = ((i.close_tick - avg_price_long)*tp_quantity_long)*(1-comission/100)

                new_row = [i.Time, entry_price_4_long,1,'Long 4', position_qty_long_4, tp_target_long, tp_quantity_long,0,0,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
                trades.loc[len(trades)] = new_row # type: ignore
                long_4_taken = True

    if tp_target_long != 0: # take profit long when there is no open hedged short trade
        if short_5_taken == False:
            if tp_long_taken == False:
                if i.close_tick>tp_target_long:
                    tp_long_taken = True
                    
                    avg_price_long = ((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long
                    tp_quantity_long = round(position_qty_long_1+position_qty_long_2+position_qty_long_3+position_qty_long_4, inp_round_qty)
                    tp_target_long = round((((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long)*(1+tp_long/100), inp_round_price)

                    tp_price_long = round(i.close_tick, inp_round_price)
                    pnl_long = ((tp_price_long - avg_price_long)*tp_quantity_long)*(1-comission/100)
                    cumulative_profit_long = pnl_long+cumulative_profit_long
                    cumulative_profit = cumulative_profit_long+cumulative_profit_short

                    new_row = [i.Time, tp_price_long,-1,'Long TP', 0, 0, 0,0,0,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,0,avg_price_short]
                    trades.loc[len(trades)] = new_row # type: ignore

                    long_1_taken = False
                    long_2_taken = False
                    long_3_taken = False
                    long_4_taken = False
                    tp_long_taken = False

                    short_5_taken = False
                    tp_short_taken = False

                    entry_price_1_long = 0
                    entry_price_2_long = 0
                    entry_price_3_long = 0
                    entry_price_4_long = 0
                    tp_price_long = 0

                    dca_grid_2_long = 0
                    dca_grid_3_long = 0
                    dca_grid_4_long = 0
                    dca_grid_5_short = 0

                    entry_price_5_short = 0
                    tp_price_short = 0

                    position_qty_long_1 = 0
                    position_qty_long_2 = 0
                    position_qty_long_3 = 0
                    position_qty_long_4 = 0

                    position_qty_short_5 = 0

                    close_resampled = 0

                    avg_price_long = 0
                    avg_price_short = 0

                    tp_target_long = 0
                    tp_target_short = 0

                    tp_quantity_long = 0
                    tp_quantity_short = 0

                    pnl_sum = 0
                    pnl_long = 0
                    pnl_short = 0

    if long_4_taken == True: # Short hedge
        if short_5_taken == False:
            if i.close_tick<dca_grid_5_short:
                entry_price_5_short = round(i.close_tick, inp_round_price)

                position_qty_short_5 = round((deposit*order_5_short/100*leverage)/(entry_price_5_short), inp_round_qty)

                tp_quantity_short = round(position_qty_short_5, inp_round_qty)
                tp_target_short = round(((entry_price_5_short)*(1-tp_short/100)), inp_round_price)

                avg_price_short = entry_price_5_short
                pnl_short = ((avg_price_short - i.close_tick)*tp_quantity_short)*(1-comission/100)

                new_row = [i.Time, entry_price_5_short,0,'Short 5',0,0,tp_quantity_long, position_qty_short_5, tp_target_short, tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
                trades.loc[len(trades)] = new_row # type: ignore
                short_5_taken = True

    if short_5_taken == True: # TP for short and SL for long
        if tp_short_taken == False:
            if i.close_tick<tp_target_short: # TP for short and SL for long
                tp_short_taken = True

                # """part to calculate long returns"""
                tp_price_long = round(i.close_tick, inp_round_price)
                pnl_long = ((tp_price_long - avg_price_long)*tp_quantity_long)*(1-comission/100)
                cumulative_profit_long = pnl_long+cumulative_profit_long
                cumulative_profit = cumulative_profit_long+cumulative_profit_short

                # """part to calculate short returns"""
                avg_price_short = ((entry_price_5_short*position_qty_short_5))/tp_quantity_short
                tp_quantity_short = round(position_qty_short_5, inp_round_qty)
                tp_target_short = round((((entry_price_5_short*position_qty_short_5)/tp_quantity_short)*(1-tp_short/100)), inp_round_price)

                tp_price_short = round(i.close_tick, inp_round_price)
                pnl_short = ((avg_price_short - tp_price_short)*tp_quantity_short)*(1-comission/100)
                cumulative_profit_short = pnl_short+cumulative_profit_short
                cumulative_profit = cumulative_profit_long+cumulative_profit_short
                
                new_row = [i.Time, tp_price_short,-1,'Short TP + Long SL', 0, 0,0,0,0,0,cumulative_profit_long,cumulative_profit_short,cumulative_profit,0,0]
                trades.loc[len(trades)] = new_row # type: ignore

                # """Reset Counters:"""
                long_1_taken = False
                long_2_taken = False
                long_3_taken = False
                long_4_taken = False
                tp_long_taken = False

                short_5_taken = False
                tp_short_taken = False

                entry_price_1_long = 0
                entry_price_2_long = 0
                entry_price_3_long = 0
                entry_price_4_long = 0
                tp_price_long = 0

                dca_grid_2_long = 0
                dca_grid_3_long = 0
                dca_grid_4_long = 0
                dca_grid_5_short = 0

                entry_price_5_short = 0
                tp_price_short = 0

                position_qty_long_1 = 0
                position_qty_long_2 = 0
                position_qty_long_3 = 0
                position_qty_long_4 = 0

                position_qty_short_5 = 0

                close_resampled = 0

                avg_price_long = 0
                avg_price_short = 0

                tp_target_long = 0
                tp_target_short = 0

                tp_quantity_long = 0
                tp_quantity_short = 0

                pnl_sum = 0
                pnl_long = 0
                pnl_short = 0

    if short_5_taken == True: # TP for long and SL for short
        if tp_short_taken == False and tp_long_taken == False:
            if i.close_tick>tp_target_long: # TP for long and SL for short
                tp_short_taken = True
                tp_long_taken = True

                # """part to calculate long returns"""
                tp_quantity_long = round(position_qty_long_1+position_qty_long_2+position_qty_long_3+position_qty_long_4, inp_round_qty)
                avg_price_long = ((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long
                tp_target_long = round((((entry_price_1_long*position_qty_long_1)+(entry_price_2_long*position_qty_long_2)+(entry_price_3_long*position_qty_long_3)+(entry_price_4_long*position_qty_long_4))/tp_quantity_long)*(1+tp_long/100), inp_round_price)

                tp_price_long = round(i.close_tick, inp_round_price)
                pnl_long = ((tp_price_long - avg_price_long)*tp_quantity_long)*(1-comission/100)
                cumulative_profit_long = pnl_long+cumulative_profit_long
                cumulative_profit = cumulative_profit_long+cumulative_profit_short

                # """part to calculate short returns"""
                tp_quantity_short = round(position_qty_short_5, inp_round_qty)
                avg_price_short = ((entry_price_5_short*position_qty_short_5))/tp_quantity_short
                tp_target_short = round((((entry_price_5_short*position_qty_short_5)/tp_quantity_short)*(1-tp_short/100)), inp_round_price)

                tp_price_short = round(i.close_tick, inp_round_price)
                pnl_short = ((avg_price_short - tp_price_short)*tp_quantity_short)*(1-comission/100)
                cumulative_profit_short = pnl_short+cumulative_profit_short
                cumulative_profit = cumulative_profit_long+cumulative_profit_short

                # trades = pd.DataFrame(columns=['time','close_tick','pos','side','position_qty_long','tp_target_long','tp_quantity_long','position_qty_short','tp_target_short','tp_quantity_short','cumulative_profit_long','cumulative_profit_short','cumulative_profit','avg_price_long','avg_price_short'])
                new_row = [i.Time, tp_price_short,-1,'Long TP + Short SL', 0, 0,0,0,0,0,cumulative_profit_long,cumulative_profit_short,cumulative_profit,0,0]
                trades.loc[len(trades)] = new_row # type: ignore

                # """Reset Counters:"""
                long_1_taken = False
                long_2_taken = False
                long_3_taken = False
                long_4_taken = False
                tp_long_taken = False

                short_5_taken = False
                tp_short_taken = False

                entry_price_1_long = 0
                entry_price_2_long = 0
                entry_price_3_long = 0
                entry_price_4_long = 0
                tp_price_long = 0

                dca_grid_2_long = 0
                dca_grid_3_long = 0
                dca_grid_4_long = 0
                dca_grid_5_short = 0

                entry_price_5_short = 0
                tp_price_short = 0

                position_qty_long_1 = 0
                position_qty_long_2 = 0
                position_qty_long_3 = 0
                position_qty_long_4 = 0

                position_qty_short_5 = 0

                close_resampled = 0

                avg_price_long = 0
                avg_price_short = 0

                tp_target_long = 0
                tp_target_short = 0

                tp_quantity_long = 0
                tp_quantity_short = 0

                pnl_sum = 0
                pnl_long = 0
                pnl_short = 0

    if show_real_drawdown == True:
        if i.signal_long==0:
            close_resampled = i.close_resampled

            new_row = [i.Time,close_resampled,0,0,0,tp_target_long,tp_quantity_long,0,tp_target_short,tp_quantity_short,cumulative_profit_long,cumulative_profit_short,cumulative_profit,avg_price_long,avg_price_short]
            trades.loc[len(trades)] = new_row # type: ignore

In [ ]:

trades = trades.set_index(trades['time'])
trades['cumulative_profit_short'] = round(trades['cumulative_profit_short'],2)
trades['cumulative_profit_long'] = round(trades['cumulative_profit_long'],2)
trades['cumulative_profit'] = round(trades['cumulative_profit'],2)
trades['cumulative_profit_per'] = round(trades['cumulative_profit']/deposit*100,2)
trades['avg_price_long'] = round(trades['avg_price_long'],inp_round_price)
trades['avg_price_short'] = round(trades['avg_price_short'],inp_round_price)
trades['number_trades_long'] = (trades['side']=='Long TP').sum()
trades['number_trades_short'] = (trades['side']=='Short TP').sum()
trades['qty'] = trades['pos']*trades['position_qty_long'] + trades['pos']*trades['position_qty_short'] # this calculations needs only for vectorbt
trades['long_pnl_usd'] = round(((trades['close_tick']-trades['avg_price_long'])*trades['tp_quantity_long'])*(1-comission/100),2)
trades['short_pnl_usd'] = round(((trades['avg_price_short'] - trades['close_tick'])*trades['tp_quantity_short'])*(1-comission/100),2)
trades['total_pnl_usd'] = round(trades['long_pnl_usd']+trades['short_pnl_usd'],2)
trades['long_drawdown_per'] = round(trades['long_pnl_usd']/deposit*100,2)
trades['short_drawdown_per'] = round(trades['short_pnl_usd']/deposit*100,2)
trades['total_drawdown_per'] = round((trades['long_pnl_usd']+trades['short_pnl_usd'])/deposit*100,2)
trades['account_balance'] = deposit+trades['total_pnl_usd']
trades['liquidation_price'] = round(((trades['account_balance'] - (trades['tp_quantity_long']*trades['avg_price_long']) + (trades['tp_quantity_short']*trades['avg_price_short']))/ ((trades['tp_quantity_long'] * (MaintenanceMarginRate/100)) + (trades['tp_quantity_short'] * (MaintenanceMarginRate/100)) - trades['tp_quantity_long'] + trades['tp_quantity_short'])),inp_round_price)
trades.loc[trades['liquidation_price'] < 0, 'liquidation_price'] = np.inf # delete negative values, becouse when value is negative it means that liqudation is inf

In [ ]:
# Print the backtest trades
print(trades.tail(20))

In [ ]:
# Save the trades to csv
trades.to_csv(dir+f'/backtest_trades_{inp_ticker}.csv',index=True)

In [ ]:
# Print strategy backtest results:
print('\n'+('-'*5)+'Backtest Results - '+inp_strategy_name+' - '+inp_ticker+('-'*5))
print('Start Date:', inp_start)
print('End Date:', inp_end)
print('Time Frame:', inp_tf)
print('Leverage:', leverage)
print('Initial Deposit (USD):', deposit,'$')
print('End Deposit (USD):', (deposit+trades['cumulative_profit'].iloc[-1]).round(2),'$')
print('Total Return (USD):',(trades['cumulative_profit'].iloc[-1]).round(2),'$')
print('Total Return (%):',(trades['cumulative_profit_per'].iloc[-1]).round(2),'%')
print('Number of Closed Trades:', (trades['number_trades_long'].iloc[-1]+trades['number_trades_short'].iloc[-1]))
print('Max Long Loss (USD):',(min(trades['long_pnl_usd'].round(2))),'$')
print('Max Short Loss (USD):',(min(trades['short_pnl_usd'].round(2))),'$')
print('Max Drawdown (USD):',(min(trades['total_pnl_usd'].round(2))),'$')
print('Max Drawdown (%):',(min(trades['total_drawdown_per'].round(2))),"%")
print('Max Drawdown Long (USD):',(min(trades['long_pnl_usd'].round(2))),'$')
print('Max Drawdown Long (%):',(min(trades['long_drawdown_per'].round(2))),"%")
print('Max Drawdown Short (USD):',(min(trades['short_pnl_usd'].round(2))),'$')
print('Max Drawdown Short (%):',(min(trades['short_drawdown_per'].round(2))),"%")
print('Max Liquidation Price:',(max(trades['liquidation_price'].round(2))),'$')
print('Min Liquidation Price:',(min(trades['liquidation_price'].round(2))),'$')
print('Max Asset Price:',(max(trades['close_tick'].round(inp_round_price))),'$')
print('Min Asset Price:',(min(trades['close_tick'].round(inp_round_price))),'$')
print('Open Profit/Loss ($):', trades['total_pnl_usd'].iloc[-1])

In [ ]:
print('\n'+('-'*5)+'Strategy Settings'+('-'*5))
print('ema len:',inp_ema_len)
print('grid 1 long - deposit (%):', order_1_long)
print('grid 2 long - deposit (%):', order_2_long)
print('grid 3 long - deposit (%):', order_3_long)
print('grid 4 long - deposit (%):', order_4_long)
print('grid 5 short - deposit (%):', order_5_short)
print('grid 1 long - dca (%):', long_1_dca_per)
print('grid 2 long - dca (%):', long_2_dca_per)
print('grid 3 long - dca (%):', long_3_dca_per)
print('grid 4 long - dca (%):', long_4_dca_per)
print('grid 5 short - dca (%):', short_5_dca_per)
print('TP Long (%):', tp_long)
print('TP Short (%):', tp_short)

In [ ]:
# Plot the strategy results suing vectorbt:
if plot_results:
    # Strategy plot:
    portfolio = vbt.Portfolio.from_orders(close=trades['close_tick'],price = trades['close_tick'], size = trades['pos'], fees=(comission/100), freq='1s')

    # Plot backtest results on chart:
    # Create a DataFrame for OHLC Candlestick:
    dff = pd.DataFrame()
    dff["Open"] = tick_resampled['open_resampled']
    dff["High"] = tick_resampled['high_resampled']
    dff["Low"] = tick_resampled['low_resampled']
    dff["Close"] = tick_resampled['close_resampled']
    # dff["Volume"] = bt_data['Volume']
    dff["Time"] = tick_resampled['Time']
    dff = dff.set_index(dff['Time'])

    cols = ['Open', 'High', 'Low', 'Close']#, "Volume"]
    ohlcv_candlestick = dff.get(cols)
    # ohlcv_wbuf = ohlcv_wbuf.astype(np.float64)
    # chart = ohlcv_wbuf.vbt.ohlc.plots(settings=dict(plot_type='candlestick'))
    # chart.show()

    # Plot the portfolio:
    fig = portfolio.plot(title=dict(text=" "+inp_strategy_name+" - Backtest Results - "+inp_ticker), subplots=['orders',('Candlestick', dict(title=' ', can_plot_groups=False, rangeslider=False, xaxis_kwargs=dict(title=' ',fixedrange=False), yaxis_kwargs=dict(title=' ',fixedrange=False))), ("price", dict(title="Cumulative Profit (USD)", yaxis_kwargs=dict(title=""))), ("price", dict(title="Cumulative Profit (%)", yaxis_kwargs=dict(title=""))), ("price", dict(title="Open PnL & Drawdown (USD)", yaxis_kwargs=dict(title=""))), ("price", dict(title="Open PnL & Drawdown (%)", yaxis_kwargs=dict(title=""))), ("price", dict(title="Liquidation Price", yaxis_kwargs=dict(title=""))) ], make_subplots_kwargs=(dict(rows=7, cols=1)))
    del portfolio

    ma_cumulative_profit=vbt.pandas_ta("EMA").run(close=trades['cumulative_profit'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_cumulative_profit, x_labels = ma_cumulative_profit.index, trace_names = ["Cumulative Profit"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=2, col=1),fig=fig)

    ma_cumulative_profit_per=vbt.pandas_ta("EMA").run(close=trades['cumulative_profit_per'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_cumulative_profit_per, x_labels = ma_cumulative_profit_per.index, trace_names = ["Cumulative Profit (%)"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=3, col=1),fig=fig)

    ma_open_pnl=vbt.pandas_ta("EMA").run(close=trades['total_pnl_usd'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_open_pnl, x_labels = ma_open_pnl.index, trace_names = ["Open PNL & Drawdown(USD)"], trace_kwargs=dict(line=dict(color="black")), add_trace_kwargs=dict(row=4, col=1),fig=fig)
    fig.add_hline(y=0, line_color="blue", row=4, col=1, line_width=2) # type: ignore

    ma_open_pnl_per=vbt.pandas_ta("EMA").run(close=trades['total_drawdown_per'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_open_pnl_per, x_labels = ma_open_pnl_per.index, trace_names = ["Open PNL & Drawdown(%)"], trace_kwargs=dict(line=dict(color="black")), add_trace_kwargs=dict(row=5, col=1),fig=fig)
    fig.add_hline(y=0, line_color="blue", row=5, col=1, line_width=2) # type: ignore

    ma_liquidation_price=vbt.pandas_ta("EMA").run(close=trades['liquidation_price'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_liquidation_price, x_labels = ma_liquidation_price.index, trace_names = ["Liquidation price"], trace_kwargs=dict(line=dict(color="black")), add_trace_kwargs=dict(row=6, col=1),fig=fig)
    ma_close_price=vbt.pandas_ta("EMA").run(close=trades['close_tick'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_close_price, x_labels = ma_close_price.index, trace_names = ["Close price"], trace_kwargs=dict(line=dict(color="blue")), add_trace_kwargs=dict(row=6, col=1),fig=fig)
    ma_close_price=vbt.pandas_ta("EMA").run(close=trades['avg_price_long'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_close_price, x_labels = ma_close_price.index, trace_names = ["Avg Long Price"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=6, col=1),fig=fig)
    ma_close_price=vbt.pandas_ta("EMA").run(close=trades['avg_price_short'], length=1).ema.round(2) # type: ignore
    scatter = vbt.plotting.Scatter(data = ma_close_price, x_labels = ma_close_price.index, trace_names = ["Avg Short Price"], trace_kwargs=dict(line=dict(color="red")), add_trace_kwargs=dict(row=6, col=1),fig=fig)

    # Plot EMAs:
    # ema_1 = round(ema(close= dff["Close"], length=inp_ema_len),inp_round_price).shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_1, x_labels = ema_1.index, trace_names = ["EMA"], trace_kwargs=dict(line=dict(color="blue")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    # ema_long_1 = round((ema_1-ema_1*long_1_dca_per/100),inp_round_price)#.shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_long_1, x_labels = ema_long_1.index, trace_names = ["EMA 1 Long"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    # ema_long_2 = round((ema_1-ema_1*long_2_dca_per/100),inp_round_price)#.shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_long_2, x_labels = ema_long_2.index, trace_names = ["EMA 2 Long"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    # ema_long_3 = round((ema_1-ema_1*long_3_dca_per/100),inp_round_price)#.shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_long_3, x_labels = ema_long_3.index, trace_names = ["EMA 3 Long"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    # ema_long_4 = round((ema_1-ema_1*long_4_dca_per/100),inp_round_price)#.shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_long_4, x_labels = ema_long_4.index, trace_names = ["EMA 4 Long"], trace_kwargs=dict(line=dict(color="green")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    # ema_short_5 = round((ema_1-ema_1*short_5_dca_per/100),inp_round_price)#.shift(1)
    # scatter = vbt.plotting.Scatter(data = ema_short_5, x_labels = ema_short_5.index, trace_names = ["EMA 5 Short"], trace_kwargs=dict(line=dict(color="red")), add_trace_kwargs=dict(row=1, col=1),fig=fig)

    ohlcv_candlestick.vbt.ohlc.plot(plot_type='Candlestick',fig=fig) # type: ignore

    fig.update_layout(xaxis_rangeslider_visible=False) # type: ignore
    fig.show() # type: ignore